In [117]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [118]:
group_features_df = pd.read_pickle ('../scripts/tmp_group_features.pkl')
technique_features_df = pd.read_pickle ('../scripts/tmp_technique_features.pkl')

In [119]:
group_features_df.head()

,group_ID,software_ID
0,G0099,[s0434]
1,G0006,"[s0017, s0025, s0119, s0026, s0121, s0002, s00..."
2,G0005,"[s0040, s0015, s0003]"
3,G0023,[s0064]
4,G0025,[s0069]


In [120]:
technique_features_df.head()

,technique_ID,platforms,tactics,data_sources,defenses_bypassed,permissions_required,mitigation_ID,detection_name,software_ID
0,T1548,"[linux, windows, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_metadata...",[],"[administrator, user]","[m1047, m1038, m1028, m1026, m1022, m1052]","[command_execution, file_metadata, file_modifi...",[]
1,T1548.002,[windows],"[defense_evasion, privilege_escalation]","[command_command_execution, process_process_cr...",[windows_user_account_control],"[administrator, user]","[m1047, m1026, m1051, m1052]","[command_execution, process_creation, process_...","[s0584, s0129, s0640, s0606, s0570, s1068, s00..."
2,T1548.004,[macos],"[defense_evasion, privilege_escalation]","[process_os_api_execution, process_process_cre...",[],"[administrator, user]",[m1038],"[os_api_execution, process_creation]",[s0402]
3,T1548.001,"[linux, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_metadata...",[],[user],[m1028],"[command_execution, file_metadata, file_modifi...","[s0401, s0276]"
4,T1548.003,"[linux, macos]","[defense_evasion, privilege_escalation]","[command_command_execution, file_file_modifica...",[],[user],"[m1028, m1026, m1022]","[command_execution, file_modification, process...","[s0154, s0281, s0279]"


# Maxium length of feature vector for each feature

In [121]:
def get_max_feature_length(df: pd.DataFrame(), id_name: str):
    for col in list (df.columns)[1:]:
        print ('{col}: {max_len}'.format(col = col, max_len = df[col].apply(lambda x: len(x)).max()))
        max_length_row_index = df[col].apply(len).idxmax()
        print ('ID with longest vector:', df.loc[max_length_row_index, id_name])


In [122]:
get_max_feature_length(group_features_df, 'group_ID')

software_ID: 46
ID with longest vector: G0016


In [123]:
get_max_feature_length(technique_features_df, 'technique_ID')

platforms: 10
ID with longest vector: T1110
tactics: 4
ID with longest vector: T1078
data_sources: 14
ID with longest vector: T1562
defenses_bypassed: 8
ID with longest vector: T1562
permissions_required: 4
ID with longest vector: T1485
mitigation_ID: 11
ID with longest vector: T1552
detection_name: 14
ID with longest vector: T1562
software_ID: 334
ID with longest vector: T1105


In [139]:
import matplotlib.colors as mcolors

def plot_sparse_matrix (df: pd.DataFrame(), feature_name:str):
    unique_strings = list(set(string for sublist in df[feature_name] for string in sublist))
    # Create a binary matrix indicating the presence of each unique string in each list
    binary_matrix = np.array([[1 if unique_string in sublist else 0 for unique_string in unique_strings] for sublist in df[feature_name]])

    # Define a custom colormap
    cmap = mcolors.ListedColormap(['white', 'black'])

    # Get the dimensions of the binary matrix
    num_rows, num_columns = binary_matrix.shape

    # Create a square heatmap with the custom colormap and equal aspect ratio
    plt.figure(figsize=(num_columns, num_rows))
    plt.imshow(binary_matrix, cmap=cmap, aspect='auto', interpolation='nearest')
    plt.title('Sparsity Map for List Column')
    plt.xlabel('Unique Strings')
    plt.ylabel('Items (Lists)')
    plt.xticks(np.arange(len(unique_strings)), unique_strings, rotation=90)
    plt.show()

In [140]:
plot_sparse_matrix (group_features_df, 'software_ID')

In [141]:
plot_sparse_matrix (technique_features_df, 'software_ID')
